In [1]:
import itertools
import os
import random
import json
import string
from json import JSONEncoder

def _default(self, obj):
    return getattr(obj.__class__, "to_json", _default.default)(obj)

_default.default = JSONEncoder().default
JSONEncoder.default = _default

from pathos import multiprocessing
import logging
import copy
from pathlib import Path
from typing import Tuple
import re
from z3.z3 import Solver, And, Or, Not, Bool, Int, sat

baselines: Path = Path("C:/Users/kisam/dbc.json")
experimental_results: Path = Path("C:/Users/kisam/dcd.json")

with open(baselines) as f:
    baselines = json.load(f)

with open(experimental_results) as f:
    experimental_results = json.load(f)

lonely_baselines = copy.deepcopy(baselines)
lonely_experimental_results = copy.deepcopy(experimental_results)

class IntRange:
    def __init__(self, lower_bound_inclusive, upper_bound_exclusive):
        self.lower_bound_inclusive = lower_bound_inclusive
        self.upper_bound_exclusive = upper_bound_exclusive

    def __contains__(self, item):
        return isinstance(item, int) and (self.lower_bound_inclusive <= item < self.upper_bound_exclusive)

    def __repr__(self):
        return f"IntRange({self.lower_bound_inclusive, self.upper_bound_exclusive})"

    def __str__(self):
        return f"[{self.lower_bound_inclusive}:{self.upper_bound_exclusive})"

    def to_json(self):
        return str(self)


for e in experimental_results:
    toks = e['original_line'].split(':')
    try:
        e['original_line'] = IntRange(int(toks[0]), int(toks[1]) + 1)
    except Exception as ex:
        e['original_line'] = []
    #print('\t'.join(["experimental", *[str(s) for s in e.values()]]).replace("\n", ""))

    if e['function_line_range'] == 'ERROR':
        e['function_line_range'] = []
    else:
        toks = e['function_line_range'].split(':')
        try:
            e['function_line_range'] = IntRange(int(toks[1]), int(toks[2]) + 1)
        except Exception as ex:
            logging.exception(f"e was {e}")
    e['presence_condition'] = str(e['presence_condition'])

print(f"We have {len(baselines)} baseline results.")
print(f"We have {len(experimental_results)} experimental results.")

We have 44 baseline results.
We have 75 experimental results.


In [2]:
import tqdm


def match_stats(baseline_result: dict, experimental_result: dict) -> Tuple:
    """
    Returns a vector of different match information.
    (a, b, c)
    a = True iff baseline and experimental have the same line number, message, and file.
    b = True iff baseline and experimental have the same message, file, and baseline is within experimental's function scope.
    c = True iff baseline's configuration is compatible with experimental's presence condition.
    """

    a = (baseline_result['sanitized_message'] == experimental_result['sanitized_message'] and \
         baseline_result['input_line'] in experimental_result['original_line'] and\
         baseline_result['input_file'].split('.')[0] == experimental_result['input_file'].split('.')[0])

    b = (baseline_result['sanitized_message'] == experimental_result['sanitized_message'] and \
         baseline_result['input_line'] in experimental_result['function_line_range'] and\
         baseline_result['input_file'].split('.')[0] == experimental_result['input_file'].split('.')[0])

    c = False

    if experimental_result['feasible'] and 'Or(None' not in experimental_result['presence_condition'] and experimental_result['presence_condition'] not in ['Or(None)', 'None'] and (a or b):  # Don't bother doing this expensive step when the file and line number are different.
        baseline_var_mapping = {}
        for var in baseline_result['configuration']:
            if var.startswith('DEF'):
                baseline_var_mapping[re.sub(r"^(DEF_.*)", r"\1", var)] = True
            elif var.startswith('UNDEF'):
                baseline_var_mapping[re.sub(r"^UN(DEF_.*)", r"\1", var)] = False
            else:
                raise RuntimeError(f"Don't know how to handle variable {var}")

        s = Solver()
        for var, val in baseline_var_mapping.items():
            var = Bool(var)
            if val:
                s.add(var)
            else:
                s.add(Not(var))

        for mat in re.findall("DEF_[a-zA-Z0-9_]+", experimental_result['presence_condition']):
            exec(f"{mat} = Bool('{mat}')")
           
        for mat in re.findall("USE_[a-zA-Z0-9_]+", experimental_result['presence_condition']):
            exec(f"{mat} = Int('{mat}')")

        while True:
            try:
                s.add(eval(experimental_result['presence_condition']))  # TODO Definitely need to do more transformation here.
                break
            except NameError as ne:
                var = re.search("name '(.*)' is not defined", str(ne))
                exec(f"{var.group(1)} = Int('{var.group(1)}')")
        c = s.check() == sat
    return a, b, c

def tupleize(func, args): return func(*args), tuple(args)

summary = {}

# Note that results depend on the order of keys in this dictionary, because once we find a match_stats for one level we do not keep searching for the next.
#  E.g., for a given report, we will first look for results with which it has a (True, True, True) report. If it has one, we do not continue searching for
#  matches for (False, True, True), (True, False, True), etc.
result_hierarchy = {(True, True, True): 0, (False, True, True): 0, (True, False, True): 0, (True, True, False): 0, (False, True, False): 0, (False, False, True): 0, (True, False, False): 0, (False, False, False): 0}

report = []
for b in tqdm.tqdm(baselines):
    # Results are (baseline, desugared, match tuple)
    results = [(b, e, match_stats(b, e)) for e in experimental_results]
    found = False
    for r in result_hierarchy.keys():
        for res in results:
            if res[2] == r:
                found = True
                result_hierarchy[r] += 1
                # -----
                # Here is where you compile information about any specific reports you need. This block of code
                # iterates through all baselines and finds the highest level of matching that is available.
                # So, for example, if you wanted to collect all of the unmatched originals, you would uncomment out this line of code:
                #
                if (r != (True, True, True) and r != (False, True, True)):
                    report.append(res[0])
                break # DO NOT DELETE THE BREAK!
        if found:
            break


100%|████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 46.27it/s]


# Results

In [3]:
# Change this if you want the reports printed out differently. For example, if above you collected unmatched reports,
#  then here, you probably want something like
#
print(json.dumps(report, indent=2))
#
_4488 = dict()
for e in report:
    key = (e['message'], e['input_file'], e['input_line'])
    if key not in _4488:
        _4488[key] = 0
    _4488[key] += 1
print(len(_4488))
print('-----------')
print(f"Number of baseline results: {len(baselines)}")
print(f"Number of desugared results: {len(experimental_results)}")
print(f"Number of feasible desugared results: {len(list(filter(lambda e: e['feasible'],experimental_results)))}")
print(f"Number of exact matches: {result_hierarchy[(True, True, True)]}")
print(f"Number of partial matches: {result_hierarchy[False, True, True]}")
print(f"Number of unmatched: {sum(v for k, v in result_hierarchy.items() if k not in [(True, True, True), (False, True, True)])}")

[
  {
    "id": "0",
    "input_file": "/targets/VarBugsPatches/BUSYBOX/199501f2a00.c",
    "input_line": 21,
    "original_line": "ERROR",
    "function_line_range": "GLOBAL:1:29",
    "message": " Null pointer passed to 2nd parameter expecting 'nonnull' ",
    "sanitized_message": " Null pointer passed to 2nd parameter expecting 'nonnull'",
    "presence_condition": null,
    "feasible": null,
    "configuration": [
      "UNDEF_ENABLE_FEATURE_MODPROBE_MULTIPLE_OPTIONS"
    ],
    "analysis_time": 0.04818582534790039,
    "desugaring_time": null,
    "get_recommended_space": true,
    "remove_errors": true,
    "verified": null,
    "warning_path": [
      [
        27,
        7,
        9,
        21
      ]
    ],
    "alarm_type": "warning"
  },
  {
    "id": "0",
    "input_file": "/targets/VarBugsPatches/BUSYBOX/eef2317b9f5.c",
    "input_line": 30,
    "original_line": "ERROR",
    "function_line_range": "GLOBAL:1:35",
    "message": " Branch condition evaluates to a garbage v

In [4]:
results = dict()

for b in baselines:
    conf = str(b.get('configuration'))
    if conf not in results:
        results[conf] = 0
    results[conf] += 1

print(f"Number of configurations with warnings: {len(results)}")
import random
while len(results.keys()) < 1000:
    results[''.join(random.choices(string.ascii_letters, k=5))] = 0

print(f"Average warnings per config is {float(sum(results.values()))/float(len(results.values()))}")

Number of configurations with warnings: 37
Average warnings per config is 0.044


In [5]:
result_hierarchy = {(True, True, True): 0, (False, True, True): 0, (True, False, True): 0, (True, True, False): 0, (False, True, False): 0, (False, False, True): 0, (True, False, False): 0, (False, False, False): 0}

for e in tqdm.tqdm(experimental_results):
    results = [(b, e, match_stats(b, e)) for b in baselines]
    found = False
    for r in result_hierarchy.keys():
        for res in results:
            if res[2] == r:
                found = True
                result_hierarchy[r] += 1
                break
        if found:
            break

print('-----------')
print(f"Number of desugared results: {len(experimental_results)}")
print(f"Number of baseline results: {len(baselines)}")
print(f"Number of exact matches: {result_hierarchy[(True, True, True)]}")
print(f"Number of partial matches: {result_hierarchy[False, True, True]}")
print(f"Number of unmatched: {sum(v for k, v in result_hierarchy.items() if k not in [(True, True, True), (False, True, True)])}")

100%|████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 84.01it/s]

-----------
Number of desugared results: 75
Number of baseline results: 44
Number of exact matches: 35
Number of partial matches: 3
Number of unmatched: 37


In [6]:
print(len(set([(e['message'], e['input_file'], e['input_line']) for e in baselines])))

42


In [7]:
import copy
baselines: Path = Path("C:/Users/kisam/bp.json")

with open(baselines) as f:
    baselines = json.load(f)
    
results = dict()
for e in baselines:
    key = (e['message'], e['input_file'], e['input_line'])
    if key not in results:
        results[key] = []
    results[key].append(e)

def subset(a,b):
    al = a['configuration']
    bl = b['configuration']
    if len(a) == len(b):
        return (False,None)
    smaller = al if len(al) < len(bl) else bl
    bigger = al if len(al) > len(bl) else bl
    allIn = True
    for x in smaller:
        if x not in bigger:
            allIn = False
            break
    if allIn:
        aa = copy.deepcopy(a)
        aa['configuration'] = smaller
        return (True, aa)
    return (False, None)

def common(a,b):
    al = a['configuration']
    bl = b['configuration']
    if len(a) != len(b):
        return (False,None)
    swapIn = []
    for x in al:
        if x not in bl:
            swap = x[2:] if x.startswith('UNDEF') else 'UN' + x
            if swap not in bl:
                return (False, None)
        else:
            swapIn.append(x)
    if len(swapIn) == len(al) -1:
        aa = copy.deepcopy(a)
        aa['configuration'] = swapIn
        return (True, aa)
    return (False, None)

def eq(a,b):
    if len(a) != len(b):
        return False
    for x in a['configuration']:
        if x not in b['configuration']:
            return False
    return True

dedupe = []
for k,v in results.items():
    #v = list of all same bug
    anyMatch = True
    print('before',v, '\n')
    while anyMatch:
        anyMatch = False
        newList = []
        i = 0
        while i < len(v):
            matched = False
            j = i + 1
            while j < len(v):
                res = subset(v[i],v[j])
                if res[0]:
                    newList.append(res[1])
                    matched = True
                    anyMatch = True
                    v.pop(j)
                    continue
                res = common(v[i],v[j])
                if res[0]:
                    newList.append(res[1])
                    matched = True
                    anyMatch = True
                    v.pop(j)
                    continue
                if eq(v[i],v[j]):
                    v.pop(j)
                    continue
                j += 1
            if not matched:
                newList.append(v[i])
            i += 1
        v = newList
    print('after',newList, '\n')
    dedupe.extend(newList)
    
with open("C:/Users/kisam/dbp.json", 'w') as f:
    json.dump(dedupe, f, indent=2)

before [{'id': '0', 'input_file': '/targets/VarBugsPatches/APACHE/1b48bb3c1af.c', 'input_line': 5, 'original_line': 'ERROR', 'function_line_range': 'GLOBAL:1:25', 'message': 'Uninitialized Variable', 'sanitized_message': 'Uninitialized Variable', 'presence_condition': None, 'feasible': None, 'configuration': ['DEF_APU_HAS_LDAP', 'DEF_APR_HAS_SHARED_MEMORY'], 'analysis_time': 0.09903883934020996, 'desugaring_time': None, 'get_recommended_space': True, 'remove_errors': True, 'verified': None, 'warning_path': [[5]], 'function': 'apr_rmm_init', 'variable': 'rmm_lock'}, {'id': '0', 'input_file': '/targets/VarBugsPatches/APACHE/1b48bb3c1af.c', 'input_line': 5, 'original_line': 'ERROR', 'function_line_range': 'GLOBAL:1:25', 'message': 'Uninitialized Variable', 'sanitized_message': 'Uninitialized Variable', 'presence_condition': None, 'feasible': None, 'configuration': ['DEF_APU_HAS_LDAP', 'UNDEF_APR_HAS_SHARED_MEMORY'], 'analysis_time': 0.09167814254760742, 'desugaring_time': None, 'get_recom

At this point in the notebook, we have a few structures.
- summary: A dictionary mapping 3-tuples corresponding to results to a list of pairs of results.
- lonely_baselines: A list of baseline results for which no matching experimental result was found.
- lonely_experimental_results: A list of experimental results for which no matching baseline was found.

# Sample

This code randomly samples a result from each classification and prints it for inspection.

In [8]:
print()

In [9]:
print(json.dumps({"summary": {str(k): len(summary[k]) for k in summary.keys()}}))

{"summary": {}}


In [10]:
import random
for k, v in filter(lambda k: (k[0][0] or k[0][1]) and not k[0][2], summary.items()): # == str((False, False, False)), summary.items()):
    print(str(k))
    print(json.dumps(random.sample(v, k=max(1, len(v))), indent=2))
    print("-----------------------------------------------")
#{k: v for k, v in summary.items() if k != str((False, False, False))}}, indent=4))
print(f"Lonely baselines: {len(lonely_baselines)}, Lonely exps: {len(lonely_experimental_results)}")

Lonely baselines: 44, Lonely exps: 75


In [11]:
print(f"Types of lonely baselines: \n" + json.dumps([s for s in sorted(lonely_baselines, key = lambda x: x['sanitized_message'])], indent=2))

Types of lonely baselines: 
[
  {
    "id": "5",
    "input_file": "/targets/VarBugsPatches/APACHE/1b48bb3c1af.c",
    "input_line": 14,
    "original_line": "ERROR",
    "function_line_range": "GLOBAL:1:25",
    "message": " 1st function call argument is an uninitialized value ",
    "sanitized_message": " 1st function call argument is an uninitialized value",
    "presence_condition": null,
    "feasible": null,
    "configuration": [
      "DEF_APR_HAS_SHARED_MEMORY",
      "DEF_APU_HAS_LDAP"
    ],
    "analysis_time": 0.034090518951416016,
    "desugaring_time": null,
    "get_recommended_space": true,
    "remove_errors": true,
    "verified": null,
    "warning_path": [
      [
        22,
        11,
        14
      ]
    ],
    "alarm_type": "warning"
  },
  {
    "id": "5",
    "input_file": "/targets/VarBugsPatches/LINUX/7acf6cd80b2.c",
    "input_line": 44,
    "original_line": "ERROR",
    "function_line_range": "GLOBAL:1:55",
    "message": " 1st function call argument i

In [12]:
print(json.dumps([e for e in experimental_results if "BUSYBOX/eef" in e['input_file']], indent=2))

[
  {
    "id": "0",
    "input_file": "/targets/VarBugsPatches/BUSYBOX/eef2317b9f5.desugared.c",
    "input_line": 985,
    "original_line": "[26:27)",
    "function_line_range": "[5:36)",
    "message": " Value stored to '__buf_738' is never read ",
    "sanitized_message": " Value stored to 'buf' is never read",
    "presence_condition": "Or(Or(And(Or(Or(Or(Not(DEF__FORTIFY_SOURCE) ,And( (DEF__FORTIFY_SOURCE) , Not(USE__FORTIFY_SOURCE > 0) )),And( (DEF__FORTIFY_SOURCE) ,And( (USE__FORTIFY_SOURCE > 0) , Not(DEF___OPTIMIZE__) ))),And( (DEF__FORTIFY_SOURCE) ,And( (USE__FORTIFY_SOURCE > 0) ,And( (DEF___OPTIMIZE__) , Not(USE___OPTIMIZE__ > 0))))))),Or(And(And((DEF__FORTIFY_SOURCE) ,And( (USE__FORTIFY_SOURCE > 0) ,And( (DEF___OPTIMIZE__) , (USE___OPTIMIZE__ > 0)))))))",
    "feasible": true,
    "configuration": {
      "DEF__FORTIFY_SOURCE": "False",
      "USE__FORTIFY_SOURCE": "1",
      "DEF___OPTIMIZE__": "True",
      "USE___OPTIMIZE__": "1"
    },
    "analysis_time": 0.08898282051